## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras import regularizers
from keras.datasets import cifar10
from keras.models import load_model
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2021057574825988941
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4963368960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3161382640087171797
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.fit_transform(y_test).toarray()

In [4]:
classifier=Sequential()
#卷積組合
classifier.add(Convolution2D(filters = 32 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',input_shape=(32,32,3),activation='relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 64 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',activation = 'relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 32 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',activation = 'relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())
#FC
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001))) 
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())



#輸出
classifier.add(Dense(10,activation='softmax'))

In [5]:
image_ge = ImageDataGenerator(
 featurewise_center=True,
 featurewise_std_normalization=True,
 rotation_range=10,
 width_shift_range=0.1,
 height_shift_range=0.1,
 shear_range=0.1,
 zoom_range=0.1,
 horizontal_flip=True,
 vertical_flip=False,
 dtype=np.float32
)

image_ge.fit(x_train)

In [8]:
classifier = load_model('./test0425.h5')

In [18]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = classifier.fit(image_ge.flow(x_train,y_train,batch_size = 25),epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
2000/2000 [==============================] - 23s 11ms/step - loss: 1.0166 - accuracy: 0.7443 - val_loss: 0.8501 - val_accuracy: 0.7824
Epoch 2/100
2000/2000 [==============================] - 23s 12ms/step - loss: 1.0134 - accuracy: 0.7466 - val_loss: 0.7773 - val_accuracy: 0.8023
Epoch 3/100
2000/2000 [==============================] - 23s 12ms/step - loss: 1.0208 - accuracy: 0.7441 - val_loss: 0.8436 - val_accuracy: 0.7792
Epoch 4/100
2000/2000 [==============================] - 23s 12ms/step - loss: 1.0244 - accuracy: 0.7441 - val_loss: 0.8857 - val_accuracy: 0.7671
Epoch 5/100
2000/2000 [==============================] - 24s 12ms/step - loss: 1.0198 - accuracy: 0.7455 - val_loss: 0.8198 - val_accuracy: 0.7906
Epoch 6/100
2000/2000 [==============================] - 22s 11ms/step - loss: 1.0185 - accuracy: 0.7463 - val_loss: 0.8131 - val_accuracy: 0.7886
Epoch 7/100
2000/2000 [==============================] - 21s 11ms/step - loss: 1.0216 - accuracy: 0.7434 - val_loss: 0

In [30]:
classifier.save('./test0425.h5')

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [23]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.55108532e-03, 1.84481469e-05, 1.10352754e-04, 9.01590884e-01,
        1.26449252e-02, 1.77490890e-07, 7.37183000e-05, 7.02636462e-05,
        8.29149187e-02, 2.51464389e-05]], dtype=float32)